## Q&A
jeonseonjin/embedding_BAAI-bge-m3<br>
<br>
## RAG
rectified_text_2023_2.txt<br>
RecursiveCharacterTextSplitter<br>
jeonseonjin/embedding_BAAI-bge-m3<br>
FAISS<br>
beomi/gemma-ko-2b<br>

# 환경설정

In [33]:
# myenv(3.12.6)
#%pip install huggingface_hub langchain langchain-community faiss-cpu sentence-transformers torch accelerate

In [34]:
#Use 3.12.4
import os
import pandas as pd
import numpy as np
from huggingface_hub import login
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.docstore.document import Document


key_path = '/Users/jaesolshin/key/HF_TOKEN.txt'
os.environ["HF_TOKEN"] = open(key_path, 'r', encoding='utf-8').read()
login(os.environ["HF_TOKEN"], add_to_git_credential=True)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (osxkeychain).
Your token has been saved to /Users/jaesolshin/.cache/huggingface/token
Login successful


# Query-Answer Function

In [35]:
# 사전정의된 Query-Answer가 담긴 테이블
qna_df = pd.read_csv('qa_data.csv')[['질문', '답변']]

qna_df['질문'] = qna_df['질문'].apply(lambda x: x.split('질문\n')[1]) # "질문\n" 제거
qna_df['답변'] = qna_df['답변'].apply(lambda x: x.split('답변\n')[1]) # "답변\n" 제거

In [36]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# SentenceTransformer 모델 로드
embedding_model = SentenceTransformer('jeonseonjin/embedding_BAAI-bge-m3')

# 쿼리 문장들에 대한 임베딩 벡터 생성
query_texts = qna_df['질문'].to_list()
query_embeddings = embedding_model.encode(query_texts)

In [37]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# query-answer 함수 정의
def qna_answer_to_query(new_query, embedding_model=embedding_model, query_embeddings=query_embeddings, top_k=1, verbose=True):
    # 쿼리 임베딩 계산
    new_query_embedding = embedding_model.encode([new_query])

    # 코사인 유사도 계산
    cos_sim = cosine_similarity(new_query_embedding, query_embeddings)
    
    # 코사인 유사도 값이 가장 큰 질문의 인덱스 찾기
    most_similar_idx = np.argmax(cos_sim)
    similarity = np.round(cos_sim[0][most_similar_idx], 2)
    
    # 가장 비슷한 질문과 답변 가져오기
    similar_query = query_texts[most_similar_idx]
    similar_answer = qna_df.iloc[most_similar_idx]['답변']
    
    if verbose == True:
        print("가장 비슷한 질문 : ", similar_query)
        print("가장 비슷한 질문의 유사도 : ", similarity)
        print("가장 비슷한 질문의 답: ", similar_answer)

    # 결과 반환
    return similar_query, similarity, similar_answer

# RAG Function

In [38]:
# Document Loaders
path = '/Users/jaesolshin/Documents/GitHub/bokbot/rectified_text_2023.txt'
loader = TextLoader(path)
documents = loader.load()

In [39]:
# Text Splitting
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter

# splitter 정의
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
    separators=["\n\n", "\n", ". ", "? ", "! "]
)

# Document 형식의 보고서를 spliiter로 분할
splitted_documents = text_splitter.split_documents(documents)

# 문장이 마침표로 시작하면 마침표 제거, 마침표로 끝나지 않으면 마침표 추가
for i in range(len(splitted_documents)):
    splitted_documents[i].page_content = splitted_documents[i].page_content.strip('. ').strip()
    if not splitted_documents[i].page_content.endswith('.'):
        splitted_documents[i].page_content += '.'

In [40]:
#분할된 문장을 벡터db에 저장
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.docstore.in_memory import InMemoryDocstore
from langchain.docstore.document import Document
from langchain import FAISS
import faiss

# SentenceTransformer 모델 로드
embedding_model = HuggingFaceEmbeddings(model_name="jeonseonjin/embedding_BAAI-bge-m3")

# SentenceTransformer 적재를 위해 Document 객체에서 텍스트 추출
report_texts = [doc.page_content for doc in splitted_documents]

# 문서 임베딩 생성
embeddings = embedding_model.embed_documents(report_texts)

# FAISS 인덱스 생성
dim = len(embeddings[0])  # 임베딩 차원
#report_index = faiss.IndexFlatIP(dim)  # 내적 기반 인덱스 생성
report_index = faiss.IndexFlatL2(dim)  # L2 거리 기반의 인덱스 생성
report_index.add(np.array(embeddings))

In [41]:
# LangChain의 FAISS와 연결
docstore = InMemoryDocstore({idx: Document(page_content=text) for idx, text in enumerate(report_texts)})
docstore_id_map = {idx: idx for idx in range(len(report_texts))}
database = FAISS(embedding_function=embedding_model, index=report_index, docstore=docstore, index_to_docstore_id=docstore_id_map)

# Query Augmentation

In [42]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain import HuggingFacePipeline

# MPS 디바이스 설정
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

#모델 설정
model_id = "beomi/gemma-ko-2b"
dtype = torch.bfloat16 if torch.backends.mps.is_available() else torch.float32

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=dtype  # torch_dtype 설정
)

# 모델을 MPS로 이동
model = model.to(device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.41it/s]


In [43]:
# Hugging Face pipeline을 사용하여 LLM 구성
hf_pipeline = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    max_length=1024,  
    temperature=0.7, 
    top_k=50,  
    top_p=0.9,
    do_sample=True, 
    truncation=True,
    device=0 if device.type == "mps" else -1  # MPS가 있으면 0, 없으면 CPU
)

# LangChain에서 Hugging Face 모델을 LLM으로 사용
huggingface_llm = HuggingFacePipeline(pipeline=hf_pipeline)

In [44]:
from langchain.chains import RetrievalQA

# Retriever 설정
retriever = database.as_retriever()  # 데이터베이스를 Retriever로 변환

get_answer = RetrievalQA.from_llm( #RetrievalQA를 초기화
    llm=huggingface_llm, #Chat models를 지정
    retriever = retriever, #Retriever를 지정
    return_source_documents=False #응답에 원본 문서를 포함할지 결정
)

def rag_answer_to_query(query):
    answer = get_answer(query)  # 답변 요청
    result_text = answer.get('result', '')  # 결과에서 'result' 키를 안전하게 가져오기

    # 특정 텍스트 패턴 제거 로직
    if 'Helpful Answer:\n' in result_text:
        parts = result_text.split('Helpful Answer:\n')
        answer_parsed = parts[1].strip() if len(parts) > 1 else parts[0].strip()
    elif 'Answer:' in result_text:
        answer_parsed = result_text.split('Answer:')[-1].strip()
    elif 'Context:' in result_text:
        answer_parsed = result_text.split('Context:')[0].strip()    
    else:
        answer_parsed = result_text.strip()  # 기본적으로 텍스트 전체를 반환

    return answer_parsed

# Combine

In [45]:
def full_answer_to_query(new_query, critical_value=0.3):

    qna_result = qna_answer_to_query(new_query, verbose=False)
    qna_similarity = qna_result[1]
    qna_answer = qna_result[2]

    if qna_similarity >= critical_value:
        msg = 'Q&A에 등재된 내용을 기반으로 답을 구합니다.\n\n'
        return msg + qna_answer
    
    else:
        msg = 'Q&A에 등재된 질문이 없어 연차보고서를 기반으로 답을 구합니다.\n\n'
        return msg + rag_answer_to_query(new_query)

# Web Client

In [46]:
# 질문에 대한 답변 생성 함수 (RetrievalQA 활용)
def chat_with_full_answer_to_query(message, history):
    # 사용자의 질문에 대해 full_answer_to_query를 사용하여 답변 생성
    response = full_answer_to_query(message)
    
    # 질문과 답변을 히스토리에 저장 (history는 대화 히스토리)
    history.append((message, response))  
    
    # Gradio가 (응답, history)를 반환해야 하므로, 대화 기록과 함께 반환
    return history, history

In [47]:
import gradio as gr

# Gradio Chatbot 인터페이스 생성
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()  # 대화 기록을 표시하는 컴포넌트
    msg = gr.Textbox(label="질문 입력")  # 질문 입력을 위한 텍스트 박스
    clear = gr.Button("대화 기록 초기화")  # 대화 기록 초기화 버튼

    # 대화가 시작될 때 실행할 동작 정의
    def clear_history():
        return []

    msg.submit(chat_with_full_answer_to_query, inputs=[msg, chatbot], outputs=[chatbot, chatbot])

    # 기록 초기화 버튼 동작 정의
    clear.click(clear_history, None, chatbot, queue=False)

# 앱 실행
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7868
Running on public URL: https://2e8d0519aed3568df1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
